In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import os
os.listdir('/content')


['.config', 'data for training CNN model.zip', 'sample_data']

In [ ]:
import zipfile
import os

zip_path = "/content/data for training CNN model.zip"
extract_path = "/content/deepfake_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Extraction done.")

✅ Extraction done.


In [ ]:
import tensorflow as tf

train_dir = "/content/deepfake_dataset/data for CNN/train"
val_dir = "/content/deepfake_dataset/data for CNN/validation"
test_dir = "/content/deepfake_dataset/data for CNN/test"

img_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=img_size, batch_size=batch_size, label_mode='binary')

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir, image_size=img_size, batch_size=batch_size, label_mode='binary')

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, image_size=img_size, batch_size=batch_size, label_mode='binary')


Found 20000 files belonging to 2 classes.
Found 6000 files belonging to 2 classes.
Found 1600 files belonging to 2 classes.


In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN

# Quantum circuit for feature map
feature_map = ZZFeatureMap(feature_dimension=4, reps=1)
var_form = RealAmplitudes(num_qubits=4, reps=1)

qc = feature_map.compose(var_form)

from qiskit.primitives import Estimator
estimator = Estimator()

qnn = EstimatorQNN(circuit=qc, estimator=estimator, input_params=feature_map.parameters,
                   weight_params=var_form.parameters)



In [ ]:
from qiskit import QuantumCircuit
print("Qiskit is working!")


Qiskit is working!


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Wrap the QNN as a torch module
quantum_model = TorchConnector(qnn)

class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 8, 3, 1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(8*62*62, 4),  # Reduce to 4 features for quantum input]
        )
        self.q_layer = quantum_model
        self.output = nn.Linear(1, 1)

    def forward(self, x):
        x = self.cnn(x)
        x = self.q_layer(x)
        x = self.output(x)
        return torch.sigmoid(x)

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Data transforms
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),  # Converts to range [0,1]
])

# Paths
train_path = "/content/deepfake_dataset/data for CNN/train"
val_path = "/content/deepfake_dataset/data for CNN/validation"
test_path = "/content/deepfake_dataset/data for CNN/test"

# Dataset loading
train_dataset = datasets.ImageFolder(train_path, transform=transform)
val_dataset = datasets.ImageFolder(val_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

# Dataloaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridModel().to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.unsqueeze(1).float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}")


Epoch 1/10, Training Loss: 0.6935
Epoch 2/10, Training Loss: 0.6928
Epoch 3/10, Training Loss: 0.6927
Epoch 4/10, Training Loss: 0.6925
Epoch 5/10, Training Loss: 0.6923
Epoch 6/10, Training Loss: 0.6920
Epoch 7/10, Training Loss: 0.6918
Epoch 8/10, Training Loss: 0.6914
Epoch 9/10, Training Loss: 0.6910
Epoch 10/10, Training Loss: 0.6906


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.unsqueeze(1).float().to(device)
        outputs = model(images)
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Validation Accuracy: 57.03%


In [ ]:

from qiskit.primitives import BaseEstimatorV2 as Estimator
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.algorithms import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator

# Define directories
train_dir = "/content/deepfake_dataset/data for CNN/train"
val_dir = "/content/deepfake_dataset/data for CNN/validation"
test_dir = "/content/deepfake_dataset/data for CNN/test"

img_size = (64, 64)
batch_size = 32

# Prepare datasets (using TensorFlow for initial loading)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=img_size, batch_size=batch_size, label_mode='binary')
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir, image_size=img_size, batch_size=batch_size, label_mode='binary')
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, image_size=img_size, batch_size=batch_size, label_mode='binary')

# QUANTUM PART (Updated for Qiskit 2.0.0)
num_qubits = 4
feature_map = ZZFeatureMap(num_qubits, reps=2)
ansatz = RealAmplitudes(num_qubits, reps=2)

# Create quantum circuit
qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

# Create QNN with new Estimator interface
estimator = Estimator()
qnn = EstimatorQNN(
    circuit=qc,
    estimator=estimator,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    input_gradients=True  # Required for hybrid training
)

# Wrap QNN in TorchConnector
quantum_model = TorchConnector(qnn)

# HYBRID MODEL (Updated dimensions)
class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Pretrained CNN backbone
        self.cnn = models.resnet50(pretrained=True)
        # Modified fully connected layers
        self.cnn.fc = nn.Sequential(
            nn.Linear(self.cnn.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_qubits)  # Must match quantum input size
        )
        # Quantum layer
        self.quantum = quantum_model
        # Final classical layer
        self.output = nn.Linear(1, 1)  # QNN outputs 1 value per sample

    def forward(self, x):
        x = self.cnn(x)
        x = self.quantum(x)  # Quantum processing
        x = self.output(x)   # Final classification
        return torch.sigmoid(x)

# PYTORCH DATA LOADING (Updated transforms)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# TRAINING SETUP
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridModel().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# TRAINING LOOP (Updated for proper device handling)
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# EVALUATION
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
        outputs = model(images)
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (/usr/local/lib/python3.11/dist-packages/qiskit/primitives/__init__.py)

In [ ]:
# CORRECT IMPORTS FOR QISKIT 2.0.0
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes

# Updated import for Estimator in Qiskit 2.0.0
try:
    from qiskit.primitives import Estimator  # Standard import
except ImportError:
    from qiskit.primitives import Sampler as Estimator  # Fallback

# For TorchConnector
try:
    from qiskit_machine_learning.connectors import TorchConnector
except ImportError:
    from qiskit_machine_learning.algorithms import TorchConnector

from qiskit_machine_learning.neural_networks import EstimatorQNN

ImportError: cannot import name 'Sampler' from 'qiskit.primitives' (/usr/local/lib/python3.11/dist-packages/qiskit/primitives/__init__.py)

In [ ]:
from qiskit import QuantumCircuit, Aer
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.

In [ ]:
!pip uninstall -y qiskit qiskit-terra qiskit-aer qiskit-ibmq-provider qiskit-machine-learning
!pip cache purge

Found existing installation: qiskit 1.0.0
Uninstalling qiskit-1.0.0:
  Successfully uninstalled qiskit-1.0.0
Found existing installation: qiskit-terra 0.46.3
Uninstalling qiskit-terra-0.46.3:
  Successfully uninstalled qiskit-terra-0.46.3
Found existing installation: qiskit-machine-learning 0.6.1
Uninstalling qiskit-machine-learning-0.6.1:
  Successfully uninstalled qiskit-machine-learning-0.6.1
Files removed: 24


In [ ]:
# If using Colab:
!python -m venv /content/qiskit_env
!source /content/qiskit_env/bin/activate

# For local machines:
# python -m venv qiskit_env
# source qiskit_env/bin/activate (Linux/Mac) or qiskit_env\Scripts\activate (Windows)

Error: Command '['/content/qiskit_env/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: /content/qiskit_env/bin/activate: No such file or directory


In [ ]:
!pip install "qiskit==1.0.0" "qiskit-machine-learning==0.6.1" "qiskit-aer==0.12.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 71.2 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit, Aer
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.

In [ ]:
!pip install "qiskit==2.0.0" "qiskit-machine-learning==0.7.0" "qiskit-aer==0.13.0"

from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_aer import Aer
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms import TorchConnector

backend = Aer.get_backend('aer_simulator')
qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    backend=backend
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 85.4 MB/s eta 0:00:00
  Attempting uninstall: qiskit
    Found existing installation: qiskit 1.0.0
    Uninstalling qiskit-1.0.0:
      Successfully uninstalled qiskit-1.0.0
  Attempting uninstall: qiskit-aer
    Found existing installation: qiskit-aer 0.12.0
    Uninstalling qiskit-aer-0.12.0:
      Successfully uninstalled qiskit-aer-0.12.0
  Attempting uninstall: qiskit-machine-learning
    Found existing installation: qiskit-machine-learning 0.6.1
    Uninstalling qiskit-machine-learning-0.6.1:
      Successfully uninstalled qiskit-machine-learning-0.6.1


ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.